In [18]:
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [19]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str

In [20]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(100)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"step3": "done"}

In [21]:
checkpointer = InMemorySaver()  

In [22]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [23]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [25]:
list(graph.get_state({"configurable": {"thread_id": 'thread-1'}}))

[{'input': 'start', 'step1': 'done'},
 ('step_2',),
 {'configurable': {'thread_id': 'thread-1',
   'checkpoint_ns': '',
   'checkpoint_id': '1f0c3645-922e-634f-8001-f2d2c063ff79'}},
 {'source': 'loop', 'step': 1, 'parents': {}},
 '2025-11-17T03:20:20.808952+00:00',
 {'configurable': {'thread_id': 'thread-1',
   'checkpoint_ns': '',
   'checkpoint_id': '1f0c3645-9224-6453-8000-0114fe3075c3'}},
 (PregelTask(id='39221fb4-4f67-77cb-59e8-8e76d5fcc671', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),),
 ()]

In [26]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c3645-922e-634f-8001-f2d2c063ff79'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-17T03:20:20.808952+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c3645-9224-6453-8000-0114fe3075c3'}}, tasks=(PregelTask(id='39221fb4-4f67-77cb-59e8-8e76d5fcc671', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c3645-9224-6453-8000-0114fe3075c3'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-17T03:20:20.804882+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'ch

In [27]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}


In [28]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c364f-711c-6d34-8003-36d804a2aa79'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-11-17T03:24:45.777001+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c364f-7110-676b-8002-e65128afeef5'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c364f-7110-676b-8002-e65128afeef5'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-17T03:24:45.771909+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c3645-922e-634f-8001-f2d2c063ff79'}}, tasks=(PregelTask(id='0920042e-d261-b65a